In [1]:
# OS System
import os
import sys
import time
from tqdm import tqdm

# Calculation
import pandas as pd
import numpy as np
import networkx as nx
from scipy.spatial import Voronoi, voronoi_plot_2dx, vertices_on_line
import math

#matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import japanize_matplotlib

#plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [2]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [3]:
RESOURCE = os.path.abspath("../Assets/Resources")
FIGURE = os.path.abspath("../Assets/Resources/Figures")

In [4]:
def get_opp_side(side):
    """
    side(1 or 2)を受け取り、反対側のサイドを返す
    e.g.) 1->2, 2->1
    超単純な作業だし、可読性の低い処理かもしれないがメモ
    
    ---
    # How it works
    
    int 1 in binary
    0 01
    
    int 2 in binary
    0 10
    
    Exclusive XOR(https://python-reference.readthedocs.io/en/latest/docs/operators/bitwise_XOR.html)
    0 ^ 0 -> 0
    0 ^ 1 -> 1
    
    So, for ex(this process needs to be done under binary format):
    0b01 ^ 0b11 -> 0b10 (which equals to 2)
    
    Finally we convert it into `int`. Put `2` as the second argument `base`.
    """
    return int(bin(side ^ 0b11), 2)

def min_dist(vertex, points):
    r = float("inf")
    for point in points:

        d = dist(point, vertex)
        
        if d < r:
            r = d
    return r     

def dist(p1, p2):
    """
    Assuming 
        p1 = (x1, y1)
        p2 = (x2, y2)
    """
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def color_by_rank(rank, echelle=5):
    """
    順位に応じて`echelle`コごとに円の色を変える(default: 5)

    e.g.: echelle = 5
    arg:  0               // 1               // 2
    rank: 0 1 2 3 4 // 5 6 7 8 9 // 10 11 12 13 14...
    """
    col = [0, 0, 0]
    # argは最大で2
    arg = min(rank//echelle, 2)
    col[arg] = 1
    return col

In [5]:
def calculate_RR(r, d_gv, d_bv, d_av, c_g, c_b, c_a, norm=True):
    """
    params:
        r: radius of empty circle
        d_gv: distance between goal and vertex
        d_bv: distance between ball and vertex
        d_av: distance between adv and vertex

        c_* : coefficient of *
        
        norm: Has to be tested.
    """
    if norm is True:
        n = np.e**-(c_g  + c_b  + c_a ) * 40
    else:
        n = 1
    return r * np.e**-(c_g * d_gv + c_b * d_bv + c_a * d_av) / n

In [6]:
rostov = pd.read_csv(f"{RESOURCE}/Rostov-20190714-181620.csv")
rostov.X = rostov.X / 525.0 * -1
rostov.Y = rostov.Y / 525.0 * -1
rostov.Time = rostov.Time - rostov.Time.min()
t = rostov.Time.unique()

In [7]:
def get_RR(opp_points,  ball, positions, radiuses, goal,  c_goal, c_ball, c_adv):
    """
    distace between:
        d_gv: goal - vertex
        d_bv: ball - vertex
        d_av:  adv - vertex
    """

    RR = np.array([])

    adv = opp_points
    
    for pos, rad in zip(positions, radiuses):
        d_gv = dist(pos, goal)
        d_bv = dist(pos, ball)
        d_av = min_dist(pos, adv)
        rr_val = calculate_RR(rad, d_gv, d_bv, d_av, c_goal, c_ball, c_adv, norm=False)
        RR = np.append(RR, rr_val)


    return RR

In [8]:
class PitchDrawer(object):
    mag = 1/52.5
    X_LIM = np.array([-52.5*mag, 52.5*mag])
    Y_LIM = np.array([-34*mag, 34*mag])
    
    def __init__(self):
        self.width = self.X_LIM.ptp()
        self.height = self.Y_LIM.ptp()
        self.penalty_spots = [(self.X_LIM.min()+11*self.mag, 0),
                              (self.X_LIM.max()-11*self.mag, 0)]
        self.ax = plt.gca()

    def plant_lawn(self, ec="white", fc="green", margin=1):
        x, y = self.X_LIM.min()*margin, self.Y_LIM.min()*margin
        width = self.width * margin
        height = self.height * margin
        lawn = patches.Rectangle(xy=(x, y), width=width, height=height, ec=ec, fc=fc, alpha=0)
        self.ax.add_patch(lawn)
        
    def add_line(self, xdata, ydata, color, linewidth=1):
        line = plt.Line2D(xdata=xdata, ydata=ydata, color=color, linewidth=linewidth)
        self.ax.add_line(line)
        
    def draw_endlines(self, color="white"):
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.min()),
                             ydata=(self.Y_LIM.min(), self.Y_LIM.max()),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.max(), self.X_LIM.max()),
                             ydata=(self.Y_LIM.min(), self.Y_LIM.max()),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.max()),
                             ydata=(self.Y_LIM.max(), self.Y_LIM.max()),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.max()),
                             ydata=(self.Y_LIM.min(), self.Y_LIM.min()),
                             color=color, linewidth=1)
        self.add_line(xdata=(0, 0),
                      ydata=(self.Y_LIM.min(), self.Y_LIM.max()),
                      color=color, linewidth=1)
        
    def draw_center_circle(self, ec="white"):
        circle = patches.Circle(xy=(0,0), radius=9.15*self.mag, fill=False, ec=ec)
        self.ax.add_patch(circle)
        
    def draw_penalty_areas(self, color="white"):
        
        """Penalty spots"""
        p_spot_left = patches.Circle(xy=self.penalty_spots[0], radius=0.01, fc=color)
        p_spot_right = patches.Circle(xy=self.penalty_spots[1], radius=0.01, fc=color)
        self.ax.add_patch(p_spot_left)
        self.ax.add_patch(p_spot_right)
        
        """Penalty Areas"""
        pa_width = (3.66+5.5+11)*self.mag
        pa_height = 16.5*self.mag
        #LEFT
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.min() + pa_height),
                             ydata=(pa_width, pa_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.min() + pa_height),
                             ydata=(-pa_width, -pa_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min() + pa_height, self.X_LIM.min() + pa_height),
                             ydata=(pa_width, -pa_width),
                             color=color, linewidth=1)
        #RIGHT
        self.add_line(xdata=(self.X_LIM.max(), self.X_LIM.max() - pa_height),
                             ydata=(pa_width, pa_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.max(), self.X_LIM.max() - pa_height),
                             ydata=(-pa_width, -pa_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.max() - pa_height, self.X_LIM.max() - pa_height),
                             ydata=(pa_width, -pa_width),
                             color=color, linewidth=1)

        """Goal Areas"""
        goal_width = (3.66+5.5)*self.mag
        goal_height = 5.5*self.mag
        #LEFT
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.min() + goal_height),
                             ydata=(goal_width, goal_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min(), self.X_LIM.min() + goal_height),
                             ydata=(-goal_width, -goal_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.min() + goal_height, self.X_LIM.min() + goal_height),
                             ydata=(goal_width, -goal_width),
                             color=color, linewidth=1)
        #RIGHT
        self.add_line(xdata=(self.X_LIM.max(), self.X_LIM.max() - goal_height),
                             ydata=(goal_width, goal_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.max(), self.X_LIM.max() - goal_height),
                             ydata=(-goal_width, -goal_width),
                             color=color, linewidth=1)
        self.add_line(xdata=(self.X_LIM.max() - goal_height, self.X_LIM.max() - goal_height),
                             ydata=(goal_width, -goal_width),
                             color=color, linewidth=1)
    def draw_penalty_arcs(self, color="white"):
        theta = np.rad2deg(np.arccos(6/10))
        left_arc = patches.Arc(self.penalty_spots[0], width=2*9.15*self.mag, height=2*9.15*self.mag,
                                 theta1=theta*-1, theta2=theta, ec=color)
        right_arc = patches.Arc(self.penalty_spots[1], width=2*9.15*self.mag, height=2*9.15*self.mag,
                                 theta1=180-theta, theta2=180-theta*-1, ec=color)
        self.ax.add_patch(left_arc)
        self.ax.add_patch(right_arc)
        
    def draw(self):
        plt.axis("scaled")
        self.ax.set_aspect("equal")
        self.ax.set_axis_off()
        self.ax.set_frame_on(False)
        self.ax.set_xbound(self.X_LIM.min(), self.X_LIM.max()*1.01)
        self.ax.set_ybound(self.Y_LIM.min()*1.01, self.Y_LIM.max())

In [43]:
RRs = {"sec":[], "mean_":[], "std_":[], "max_":[], "n":[]}

X_NORM_LIMITS = (-1.0, 1.0)
Y_NORM_LIMITS = (-68/105, 68/105)
corners = [
    (min(X_NORM_LIMITS), max(Y_NORM_LIMITS)),
    (max(X_NORM_LIMITS), max(Y_NORM_LIMITS)),
    (max(X_NORM_LIMITS), min(Y_NORM_LIMITS)),
    (min(X_NORM_LIMITS), min(Y_NORM_LIMITS))
]
lines = [
    (corners[0], corners[1]),
    (corners[1], corners[2]),
    (corners[2], corners[3]),
    (corners[3], corners[0])
]


with tqdm(t, total=len(t)) as pbar:
    for sec in pbar:  
        df = rostov[rostov.Time==sec]
        observed_points = df[df.Side==2][["X", "Y"]].values
        opp_points = df[df.Side==1][["X", "Y"]].values
        ball = df[df.Side==0][["X", "Y"]].values.reshape(2, )

        vor = Voronoi(observed_points, qhull_options="Qz")

        points_on_line, seg_dict = vertices_on_line(vor, lines, True) 

        vertices = [v for v in vor.vertices]
        vertices.extend(points_on_line)

        pos = np.array([])
        rad = np.array([])

        kwargs = {"marker": ">", "point_size": 10, "fc": "blue", "seg": seg_dict}
        voronoi_plot_2dx(vor, show_vertices=False, **kwargs)

        for vertex in vertices:
            #ピッチ内のボロノイ頂点に限る + ピッチのライン上
            if (abs(vertex[0])<=max(X_NORM_LIMITS)) & (abs(vertex[1])<=max(Y_NORM_LIMITS)):

                #最近傍点の半径
                r = min_dist(vertex, observed_points)

                pos = np.append(pos, vertex)
                rad = np.append(rad, r)

        pos = pos.reshape(-1, 2)
        # RRの計算
        my_goal = (-1, 0)

        rr = get_RR(opp_points, ball, pos, rad, my_goal, 1, 1, 1)
        df_rr = pd.DataFrame({"X":pos[:, 0], "Y":pos[:, 1], "R":rad, "RR":rr})

        df_rr.loc["ball"] = {"X":ball[0], "Y":ball[1], "R":None, "RR":None}

        mean_RR =df_rr["RR"].mean()
        std_RR = df_rr["RR"].std()
        max_RR = df_rr["RR"].max()

        RRs["sec"].append(round(sec, 2))
        RRs["mean_"].append(mean_RR)
        RRs["std_"].append(std_RR)
        RRs["max_"].append(max_RR)
        RRs["n"].append(len(df_rr["RR"]))

        ax = plt.gca()

        #plot
        #ax.set_xlim(X_NORM_LIMITS)
        #ax.set_ylim(Y_NORM_LIMITS)
        #ax.set_aspect("equal")


        #ballのプロット
        ax.scatter(ball[0], ball[ 1], s=120, marker="H", color="none", facecolors="white", edgecolors="black", linewidths=1.5)

        # 観測選手は_plotutils.py中で描画

        # 敵選手
        ax.scatter(opp_points[:, 0], opp_points[:, 1], s=100, marker="<", color="red", edgecolors="black")

        # 自陣ゴール
        ax.scatter(my_goal[0], my_goal[1], s=20, marker=".", color="black")

        drawer = PitchDrawer()
        drawer.plant_lawn(margin=1)
        drawer.draw_endlines(color="black")
        drawer.draw_center_circle(ec="black")
        drawer.draw_penalty_areas(color="black")
        drawer.draw_penalty_arcs(color="black")
        drawer.draw()

        # 円
        rank = 0

        df_rr = df_rr.sort_values("RR", ascending=False)

        for x, y, rr, r in zip(df_rr["X"], df_rr["Y"], df_rr["RR"], df_rr["R"]):
            # 色の設定
            col = color_by_rank(rank)

            # 円の描画
            alpha = rr * 2
            c_face = patches.Circle(xy=(x, y), radius=rr, fc=col, ec=col, alpha=alpha)
            c_edge = patches.Circle(xy=(x, y), radius=rr, ec=col, fill=False)
            ax.add_patch(c_face)
            ax.add_patch(c_edge)

            rank += 1  
        ax.set_title(f"{sec:.2f}, RR: {mean_RR:.4f}±{std_RR:.4f} (max: {max_RR:.4f}) \n c_g:1, c_b: 1, c_a: 1")
        plt.savefig(f"./figures/0/{sec:.2f}.png")
        #plt.savefig(f"./figures/r_equal_lec_size/{sec:.2f}.png")
        #plt.show()
        plt.close()

        pbar.update()

100%|██████████| 896/896 [08:09<00:00,  2.05it/s]  


In [29]:
def trace(data, source, time, name, color):
    data = append_trace_mean_and_std(data, source, time, name, fillcolor=color)
    data = append_trace_max(data, source, time, name)
    return data
    
def append_trace_mean_and_std(data, source, time, name, fillcolor):
    fillcolor_a = f"rgba({fillcolor[0]}, {fillcolor[1]}, {fillcolor[2]}, {fillcolor[3]})"
    fillcolor = f"rgb({fillcolor[0]}, {fillcolor[1]}, {fillcolor[2]})"
    
    trace_upper = go.Scatter(
        name = "upper_" + name,
        x = time,
        y = source["mean_"] + source["std_"],

        line=dict(width=0, color="#424242"),
        fillcolor=fillcolor_a,
        fill = "tonexty"
    )

    trace_mean = go.Scatter(
        name = "trace_" + name,
        x = time,
        y = source["mean_"],

        line=dict(color=fillcolor, width=0.4),
        fillcolor=fillcolor_a,
        fill = "tonexty"
    )

    trace_lower = go.Scatter(
        name = "lower_" + name,
        x = time,
        y = source["mean_"] - source["std_"],

        line=dict(width=0, color="#424242")
    )    
    data.extend([trace_lower, trace_mean, trace_upper])
    return data

def append_trace_max(data, source, time, name):
    # Max 
    trace_max = go.Scatter(
        name = "max_" + name,
        x = time,
        y = source["max_"],
        #mode = "markers"
    )
    data.append(trace_max)
    return data
    

In [33]:
a = []
for i in range(len(RRs["max_"])):
    if (i<2)or(i>=len(RRs["max_"])-2):
        a.append(RRs["max_"][i])
        continue
        
    a.append((RRs["max_"][i-2] + RRs["max_"][i-1] +  RRs["max_"][i] +  RRs["max_"][i+1] +  RRs["max_"][i+2])/5)

In [34]:
RRs["max_"] = a

In [35]:
stats = pd.DataFrame(RRs)

In [36]:
source = stats

time = source.sec

data = []
data = trace(data, stats, time, "Japan", color=(31, 19, 231, 0.2))
#data = trace(data, away_stats[0], time, "away", color=(239, 31, 36, 0.2))
layout = go.Layout(xaxis=dict(range=[0, 14]), yaxis=dict(range=[0, 0.15]))
fig = go.Figure(data, layout)
py.iplot(fig, filename="basic-line")

In [10]:
for c_g in [1, 2]:
            for c_b in [1, 2]:
                for c_a in [1, 2]:
                    if c_g==c_b==c_a:
                        continue
                    os.makedirs(f"./figures/{c_g:1d}{c_b:1d}{c_a:1d}/",exist_ok=True)